# PCC

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import warnings
warnings.filterwarnings("ignore")


# =============================================================================
# 1. TARGET CONFIG
# =============================================================================
TARGETS = [
    {"name": "Electrical_Conductivity_σ_S_cm_", "col": "σ"},
    {"name": "Thermal_Conductivity_κ_W_m-K_", "col": "κ"},
    {"name": "Seebeck_Coefficient_S_μV_K_", "col": "S"},
    {"name": "Figure_of_Merit_zT", "col": "zT"}
]

# =============================================================================
# 2. GLOBAL STYLE
# =============================================================================
plt.rcParams.update({
    "font.family": "Arial",
    "font.weight": "bold",
    "axes.linewidth": 8.0,
    "xtick.direction": "out",
    "ytick.direction": "out",
    "savefig.transparent": True,
    "savefig.dpi": 300
})

# =============================================================================
# 3. DIRECTORIES
# =============================================================================
MAIN_ROOT_DIR = "FINAL_RESULTS"
FIGURES_DIR = "Figures"
os.makedirs(FIGURES_DIR, exist_ok=True)

# =============================================================================
# 4. LABELS
# =============================================================================
PROPERTY_LABELS = {
    "σ": "Electrical Conductivity σ (S/cm)",
    "κ": "Thermal Conductivity κ (W/m·K)",
    "S": "Seebeck Coefficient S (μV/K)",
    "zT": "Figure of Merit (zT)"
}

def get_formal_name(col):
    return PROPERTY_LABELS.get(col, col)

# =============================================================================
# 5. MAIN LOOP
# =============================================================================
for target in TARGETS:

    t_name = target["name"]
    t_col = target["col"]

    base_dir = f"{MAIN_ROOT_DIR}/{t_name}"
    data_path = f"{base_dir}/processed_data_{t_name}.csv"
    rfe_path = f"{base_dir}/00_Common_Analysis/RFE_Feature_Selection_Details.xlsx"
    save_dir = f"{FIGURES_DIR}/{t_name}/Analysis"
    os.makedirs(save_dir, exist_ok=True)

    if not os.path.exists(data_path) or not os.path.exists(rfe_path):
        print(f"❌ Missing files for {t_col}")
        continue

    # -------------------------------------------------------------------------
    # LOAD DATA
    # -------------------------------------------------------------------------
    df = pd.read_csv(data_path)

    rfe_df = pd.read_excel(rfe_path)
    selected_features = rfe_df.loc[
        rfe_df["Is_Selected"] == True, "Feature_Name"
    ].tolist()

    selected_features = [f for f in selected_features if f in df.columns]
    if t_col not in df.columns:
        raise ValueError(f"Target {t_col} missing in data")

    # -------------------------------------------------------------------------
    # PCC CALCULATION (SELECTED FEATURES + TARGET ONLY)
    # -------------------------------------------------------------------------
    corr_vars = selected_features + [t_col]
    corr = df[corr_vars].corr(method="pearson")
    n = corr.shape[0]

    # -------------------------------------------------------------------------
    # FIGURE
    # -------------------------------------------------------------------------
    fig, ax = plt.subplots(figsize=(12, 11))
    cmap = plt.get_cmap("RdYlGn")

    mask = np.triu(np.ones_like(corr, dtype=bool))
    corr_display = corr.where(~mask)

    im = ax.imshow(
        corr_display,
        cmap=cmap,
        vmin=-1,
        vmax=1,
        extent=[-0.5, n - 0.5, n - 0.5, -0.5]
    )

    # -------------------------------------------------------------------------
    # DRAW MATRIX
    # -------------------------------------------------------------------------
    for i in range(n):
        for j in range(n):

            val = corr.iloc[i, j]

            # LOWER TRIANGLE → BOX WITH THICK WHITE GRID
            if i > j:
                ax.add_patch(
                    patches.Rectangle(
                        (j - 0.5, i - 0.5),
                        1, 1,
                        fill=False,
                        edgecolor="white",
                        lw=5.0   # ⬅ thicker grid lines
                    )
                )

            # UPPER TRIANGLE → PIE (FINITE ONLY)
            elif i < j and np.isfinite(val):
                frac = min(abs(val), 1.0)
                color = cmap((val + 1) / 2)

                ax.pie(
                    [frac, 1 - frac],
                    colors=[color, "white"],
                    radius=0.42,
                    center=(j, i),
                    startangle=90,
                    counterclock=False,
                    wedgeprops=dict(
                        width=0.42,
                        edgecolor="black",
                        linewidth=2.8   # ⬅ thicker pie edge
                    )
                )

            # DIAGONAL → FEATURE NAME
            elif i == j:
                ax.text(
                    j, i,
                    corr.columns[i],
                    ha="center",
                    va="center",
                    fontsize=24,
                    weight="bold",
                    color="red"
                )

    # -------------------------------------------------------------------------
    # AXES CLEANUP
    # -------------------------------------------------------------------------
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(-0.5, n - 0.5)
    ax.set_ylim(n - 0.5, -0.5)

    # -------------------------------------------------------------------------
    # COLORBAR (CLEAN & THIN)
    # -------------------------------------------------------------------------
    cbar = plt.colorbar(im, ax=ax, fraction=0.04, pad=0.04, aspect=50)

    cbar.set_ticks([-1, -0.5, 0, 0.5, 1])
    cbar.ax.tick_params(
        labelsize=20,
        width=1.2,
        length=6
    )
    cbar.set_label(
        "Pearson Correlation Coefficient",
        size=26,
        weight="bold"
    )
    cbar.outline.set_visible(False)

    # -------------------------------------------------------------------------
    # TITLE & SAVE
    # -------------------------------------------------------------------------
    ax.set_title(
        f"{get_formal_name(t_col)}",
        fontsize=40,
        weight="bold",
        pad=25
    )

    plt.tight_layout()
    plt.savefig(f"{save_dir}/PCC_Asymmetric_Recomputed.tif", dpi=300)
    plt.close()

    print(f"✅ PCC recomputed & plotted for {t_col}")

print("\n🎯 ALL PCC HEATMAPS GENERATED SUCCESSFULLY.")


✅ PCC recomputed & plotted for σ
✅ PCC recomputed & plotted for κ
✅ PCC recomputed & plotted for S
✅ PCC recomputed & plotted for zT

🎯 ALL PCC HEATMAPS GENERATED SUCCESSFULLY.


In [2]:
for target in TARGETS:
    for folder in ["Analysis", "Parity", "Metrics", "SHAP"]:
        os.makedirs(f"{FIGURES_DIR}/{target['name']}/{folder}", exist_ok=True)

# RFE 

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Selection from RdYlGn (Forest Green)
RDYLGN_GREEN = plt.get_cmap('RdYlGn')(0.9) 

# Scientific Label Mapping
PROPERTY_LABELS = {
    "σ": "Electrical Conductivity σ (S/cm)",
    "κ": "Thermal Conductivity κ (W/m·K)",
    "S": "Seebeck Coefficient S (μV/K)",
    "zT": "Figure of Merit (zT)"
}

# --- STANDARDIZED PLOT AREA COORDINATES ---
PLOT_BOX = [0.15, 0.15, 0.55, 0.70]

for target in TARGETS:
    t_col = target['col']
    t_name_safe = target['name'] 
    formal_name = PROPERTY_LABELS.get(t_col, t_col.replace('_', ' ')) 
    
    rfe_path = f"{MAIN_ROOT_DIR}/{t_name_safe}/00_Common_Analysis/RFE_Plot_Data.xlsx"
    save_path = f"{FIGURES_DIR}/{t_name_safe}/Analysis"
    
    if os.path.exists(rfe_path):
        df_rfe = pd.read_excel(rfe_path)
        
        # Initialize Figure
        fig = plt.figure(figsize=(20, 12))
        ax = fig.add_axes(PLOT_BOX)
        
        # --- 1. ENHANCED GRID ---
        ax.grid(which='major', linestyle='-', linewidth=2.5, color='gray', alpha=0.3, zorder=0)
        ax.grid(which='minor', linestyle=':', linewidth=1.5, color='gray', alpha=0.15, zorder=0)
        
        # --- 2. PLOTTING ---
        ax.plot(df_rfe['Num_Features'], df_rfe['CV_R2_Score'], 
                 color=RDYLGN_GREEN, marker='o', markersize=25, 
                 linewidth=8, label='CV R² Score', 
                 markerfacecolor='white', markeredgewidth=5, zorder=3)
        
        opt_idx = df_rfe['CV_R2_Score'].idxmax()
        opt_feat = int(df_rfe.loc[opt_idx, 'Num_Features'])
        ax.axvline(x=opt_feat, color='red', linestyle='--', linewidth=6, 
                    label=f'Optimal: {opt_feat} Features', zorder=2)
        
        # --- 3. DYNAMIC Y-LIMIT ---
        data_max = df_rfe['CV_R2_Score'].max()
        data_min = df_rfe['CV_R2_Score'].min()
        ax.set_ylim(data_min - (data_max - data_min) * 0.05, data_max + (data_max - data_min) * 0.30)

        # --- 4. INTEGER X-AXIS TICKS ---
        # Forces the x-axis to only show whole numbers (integers)
        ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

        # --- 5. STANDARDIZED SPINES (8pt) ---
        for side in ax.spines:
            ax.spines[side].set_linewidth(8)
            ax.spines[side].set_edgecolor('black')

        # --- 6. ENHANCED TICKS ---
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
        
        ax.tick_params(which='both', direction='in', top=True, right=True, width=6, pad=10)
        ax.tick_params(which='major', length=20, labelsize=40)
        ax.tick_params(which='minor', length=12, width=4)
        
        # --- 7. LABELS & TITLE ---
        ax.set_xlabel("Number of Features", labelpad=10, fontweight='bold', fontsize=40)
        ax.set_ylabel("Cross-Validation R² Score", labelpad=10, fontweight='bold', fontsize=40)
        ax.set_title(f"{formal_name}", pad=20, fontsize=44, fontweight='bold')
        
        # --- 8. LEGEND PLACEMENT (Away from Ticks) ---
        # Using bbox_to_anchor to "push" the legend 2% away from the spines
        if t_col == 'κ':
            # Upper Right: Pushed slightly left and down from the top-right corner
            leg_loc = 'upper right'
            bbox = (0.98, 0.98)
        else:
            # Upper Left: Pushed slightly right and down from the top-left corner
            leg_loc = 'upper left'
            bbox = (0.02, 0.98)
        
        ax.legend(loc=leg_loc, 
                  bbox_to_anchor=bbox,
                  frameon=True, 
                  facecolor='white', 
                  edgecolor='black',
                  framealpha=1,
                  prop={'size': 26, 'weight': 'bold'})
        
        # --- 9. SAVE ---
        plt.savefig(f"{save_path}/RFE_Optimization.tif", dpi=300, transparent=True, bbox_inches='tight')
        plt.close()

print(f"✅ RFE Plots generated with Integer X-ticks, Grids, and safe Legend positioning.")

✅ RFE Plots generated with Integer X-ticks, Grids, and safe Legend positioning.


# Bar Plot for Metrices

In [4]:
import matplotlib.ticker as ticker
from matplotlib import cm

# Scientific Label Mapping for proper units
PROPERTY_LABELS = {
    "σ": "Electrical Conductivity σ (S/cm)",
    "κ": "Thermal Conductivity κ (W/m·K)",
    "S": "Seebeck Coefficient S (μV/K)",
    "zT": "Figure of Merit (zT)"
}

# Clean Y-axis titles
METRIC_DISPLAY_NAMES = {
    "R²": "R² Score",
    "MAE": "MAE",
    "RMSE": "RMSE",
    "r-value": "r-value"
}

for target in TARGETS:
    t_col = target['col']
    t_name_safe = target['name'] 
    formal_name = PROPERTY_LABELS.get(t_col, t_col.replace('_', ' ')) 
    
    # Extract unit string for Y-label
    unit_str = ""
    if '(' in formal_name:
        unit_str = formal_name.split('(')[-1].replace(')', '')
    
    sum_path = f"{MAIN_ROOT_DIR}/{t_name_safe}/02_Optimized_Optuna/evaluation_summary.xlsx"
    
    if os.path.exists(sum_path):
        df_s = pd.read_excel(sum_path)
        
        # Sort so the best R2 model is first
        df_s = df_s.sort_values("R²_Mean", ascending=False)
        
        for metric in ['R²', 'MAE', 'RMSE', 'r-value']:
            m_col, s_col = f"{metric}_Mean", f"{metric}_Std"
            
            # --- FORCE SQUARE ASPECT RATIO ---
            fig, ax = plt.subplots(figsize=(16, 12))
            
            # Create Viridis colors based on number of models
            viridis_colors = cm.viridis(np.linspace(0, 0.8, len(df_s)))
            
            # 1. Bar Plot
            bars = ax.bar(df_s['Model'], df_s[m_col], color=viridis_colors, 
                          edgecolor='black', linewidth=4, zorder=3)
            
            # 2. Error Bars
            ax.errorbar(df_s['Model'], df_s[m_col], yerr=df_s[s_col], 
                        fmt='none', ecolor='black', elinewidth=5, capsize=12, zorder=4)
            
            # 3. Two-Line Mean ± Std Dev Labels
            for i, bar in enumerate(bars):
                yval = df_s.iloc[i][m_col]
                err = df_s.iloc[i][s_col]
                
                # Logic: Create a label with a newline character \n
                label_text = f"{yval:.3f}\n±{err:.3f}"
                
                # Position text above the error bar
                # The y-offset is calculated to be slightly above the cap
                ax.text(bar.get_x() + bar.get_width()/2, yval + err + (ax.get_ylim()[1]*0.01), 
                        label_text, 
                        ha='center', va='bottom', fontsize=26, fontweight='bold', 
                        linespacing=1.2)

            # --- ENHANCED GRID & TICKS ---
            ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
            plt.grid(which='major', axis='y', linestyle='-', linewidth=2.0, color='gray', alpha=0.3, zorder=1)
            plt.grid(which='minor', axis='y', linestyle=':', linewidth=1.0, color='gray', alpha=0.15, zorder=1)
            
            # Remove X ticks but keep labels; maintain Y ticks
            ax.tick_params(axis='x', which='both', length=0, pad=10) 
            ax.tick_params(axis='y', which='both', direction='in', right=True, width=4, pad= 10,)
            ax.tick_params(axis='y', which='major', length=12)
            
            # Labeling
            plt.xticks(rotation=45, ha='right', fontweight='bold', fontsize=32)
            plt.yticks(fontsize=28)
            
            # Y-label Logic
            base_y_label = METRIC_DISPLAY_NAMES.get(metric, metric)
            if metric in ['MAE', 'RMSE'] and unit_str:
                plt.ylabel(f"{base_y_label} ({unit_str})", fontweight='bold',fontsize=40)
            else:
                plt.ylabel(base_y_label, fontweight='bold', fontsize=36)
                
            plt.title(f"{formal_name}", pad=20, fontweight='bold', fontsize=36)
            
            # Add extra headroom on Y-axis for the 2-line labels
            current_ylim = ax.get_ylim()
            ax.set_ylim(current_ylim[0], current_ylim[1] * 1.15)
            
            plt.tight_layout()
            plt.savefig(f"{FIGURES_DIR}/{t_name_safe}/Metrics/Bar_Chart_{metric}.tif", dpi=300, transparent=True)
            plt.close()

print(f"🚀 Viridis-themed bar charts with 2-line labels saved for all targets.")

🚀 Viridis-themed bar charts with 2-line labels saved for all targets.


# SHAP Summary Plot

In [5]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import logging
import shap
from matplotlib.colors import LinearSegmentedColormap

# --- 1. HELPER FUNCTIONS ---
def clean_filename(name):
    return re.sub(r'[^\w\-]', '_', str(name))

def get_formal_name(target_col):
    PROPERTY_LABELS = {
        "σ": "Electrical Conductivity σ (S/cm)",
        "κ": "Thermal Conductivity κ (W/m·K)",
        "S": "Seebeck Coefficient S (μV/K)",
        "zT": "Figure of Merit (zT)"
    }
    return PROPERTY_LABELS.get(target_col, target_col)

def get_target_colorbar_label(target_col):
    TARGET_LABELS = {
        "zT": "zT value",
        "σ": "σ value",
        "κ": "κ value",
        "S": "S value"
    }
    return TARGET_LABELS.get(target_col, "Target value")


# --- 2. MAIN PLOTTING LOOP ---
for target in TARGETS:
    t_name = target['name']
    t_col = target['col']
    excel_path = f"{MAIN_ROOT_DIR}/{t_name}/02_Optimized_Optuna/FINAL_RESULTS.xlsx"
    shap_save_dir = f"{FIGURES_DIR}/{t_name}/SHAP"
    os.makedirs(shap_save_dir, exist_ok=True)
    
    if os.path.exists(excel_path):
        try:
            df_dep_all = pd.read_excel(excel_path, sheet_name='SHAP_Dependence_All')
            unique_features = df_dep_all['Feature_Name'].unique()
            num_features = len(unique_features)
            
            summary_list = []
            feature_val_list = []
            for feat in unique_features:
                feat_slice = df_dep_all[df_dep_all['Feature_Name'] == feat]
                summary_list.append(feat_slice['SHAP_Value'].values)
                feature_val_list.append(feat_slice['Feature_Value'].values)
            
            shap_matrix = np.array(summary_list).T
            feature_matrix = np.array(feature_val_list).T
            
            # --- 2. SHAP SUMMARY PLOT ---
            dynamic_height = max(18, num_features * 2.2)
            fig = plt.figure(figsize=(26, dynamic_height))
            
            shap.summary_plot(
                shap_matrix,
                feature_matrix,
                feature_names=unique_features,
                show=False,
                cmap=plt.get_cmap('viridis'),
                alpha=0.6,
                plot_size=None
            )
            
            ax_sum = plt.gca()
            
            # --- COLORBAR & AXIS FONT SIZE ADJUSTMENT (✔ FIXED HERE) ---
            for ax in fig.get_axes():
                if ax != ax_sum:
                    ax.tick_params(
                        labelsize=34, width=4, length=12,
                        labelcolor='black', pad=10
                    )
                    ax.set_ylabel(
                        f'Feature Value',   # ✅ FIX
                        fontsize=44,
                        fontweight='bold',
                        labelpad=45,
                        color='black'
                    )
                    for label in ax.get_yticklabels():
                        label.set_fontweight('bold')
                        label.set_family('Arial')

            for child in ax_sum.get_children():
                if isinstance(child, plt.matplotlib.collections.PathCollection):
                    child.set_sizes([500])
                    child.set_edgecolor('black')
                    child.set_linewidth(2.5)

            ax_sum.tick_params(
                axis='x', direction='out',
                width=5, length=20,
                labelsize=40, labelcolor='black', pad=15
            )
            ax_sum.tick_params(
                axis='y', direction='out',
                width=5, length=10,
                labelsize=44, labelcolor='black'
            )
            
            ax_sum.xaxis.set_minor_locator(ticker.AutoMinorLocator())
            ax_sum.tick_params(
                axis='x', which='minor',
                direction='out', width=5, length=12, color='black'
            )

            for spine in ax_sum.spines.values():
                spine.set_visible(True)
                spine.set_linewidth(8)
                spine.set_edgecolor('black')
            
            plt.title(
                f"SHAP Summary: {get_formal_name(t_col)}",
                fontsize=56, pad=30,
                fontweight='bold', color='black'
            )
            plt.xlabel(
                "SHAP Value (Impact on Prediction)",
                fontsize=56, fontweight='bold',
                labelpad=20, color='black'
            )
            
            plt.setp(
                ax_sum.get_yticklabels(),
                fontweight='bold',
                family='Arial',
                color='black'
            )
            ax_sum.yaxis.labelpad = 60

            plt.savefig(
                f"{shap_save_dir}/Summary_Beeswarm_HD.tif",
                dpi=300, transparent=True,
                bbox_inches='tight', pad_inches=0.5
            )
            plt.close()

            # --- 3. DEPENDENCE PLOTS ---
            for feat in unique_features:
                sub = df_dep_all[df_dep_all['Feature_Name'] == feat]
                
                fig, ax = plt.subplots(figsize=(15, 10.5))
                
                scatter = ax.scatter(
                    sub['Feature_Value'],
                    sub['SHAP_Value'],
                    c=sub['Target_True_Value'],
                    cmap='viridis',
                    s=800,
                    edgecolor='black',
                    linewidth=3.5,
                    alpha=0.8,
                    zorder=3
                )
                
                ax.axhline(
                    0, color='black',
                    linewidth=6, linestyle='--',
                    alpha=0.9, zorder=2
                )
                
                ax.tick_params(
                    which='major', direction='in',
                    width=6, length=22,
                    labelsize=38, labelcolor='black', pad=15
                )
                ax.tick_params(
                    which='minor', direction='in',
                    width=4, length=12, labelcolor='black'
                )
                ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
                ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
                
                plt.grid(
                    which='major', linestyle='-',
                    linewidth=3, color='gray',
                    alpha=0.4, zorder=1
                )
                
                plt.xlabel(
                    f"Feature Value: {feat}",
                    fontweight='bold',
                    labelpad=10,
                    fontsize=44,
                    color='black'
                )
                plt.ylabel(
                    "SHAP Value",
                    fontweight='bold',
                    labelpad=10,
                    fontsize=44,
                    color='black'
                )
                plt.title(
                    f"Dependence: {feat}",
                    pad=20,
                    fontweight='bold',
                    fontsize=52,
                    color='black'
                )
                
                for spine in ax.spines.values():
                    spine.set_linewidth(8)
                    spine.set_edgecolor('black')
                
                cbar = plt.colorbar(
                    scatter, ax=ax,
                    fraction=0.046, pad=0.06)
                
                cbar.ax.tick_params(
                    direction='out',
                    width=4, length=14,
                    labelsize=34, labelcolor='black', pad=10
                )
                cbar.set_label(
                    get_target_colorbar_label(t_col),
                    fontweight='bold',
                    size=44,
                    labelpad=5,
                    color='black'
                )
                
                plt.savefig(
                    f"{shap_save_dir}/Dependence_{clean_filename(feat)}.tif",
                    dpi=300, transparent=True,
                    bbox_inches='tight', pad_inches=0.5
                )
                plt.close()
                
        except Exception as e:
            logging.error(f"❌ SHAP Formatting Error for {t_name}: {e}")

print("✅ Final Ultra-Bold Plots saved with target-named colorbars.")


✅ Final Ultra-Bold Plots saved with target-named colorbars.


In [6]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import logging
import shap

# =============================================================================
# MAIN LOOP
# =============================================================================
for target in TARGETS:
    t_name = target["name"]
    t_col = target["col"]

    excel_path = f"{MAIN_ROOT_DIR}/{t_name}/02_Optimized_Optuna/FINAL_RESULTS.xlsx"
    shap_save_dir = f"{FIGURES_DIR}/{t_name}/SHAP"
    os.makedirs(shap_save_dir, exist_ok=True)

    if not os.path.exists(excel_path):
        continue

    try:
        df_dep_all = pd.read_excel(excel_path, sheet_name="SHAP_Dependence_All")

        # --- CALCULATE IMPORTANCE ---
        importance_data = (
            df_dep_all.groupby("Feature_Name")["SHAP_Value"]
            .apply(lambda x: np.mean(np.abs(x)))
            .reset_index()
        )
        importance_data.columns = ["Feature_Name", "Global_Importance"]
        importance_data = importance_data.sort_values(by="Global_Importance", ascending=True)

        sorted_features = importance_data["Feature_Name"].tolist()
        importance_vals_sorted = importance_data["Global_Importance"].values
        num_features = len(sorted_features)

        # --- RECONSTRUCT MATRICES ---
        shap_list, feature_list = [], []
        for feat in sorted_features:
            sub = df_dep_all[df_dep_all["Feature_Name"] == feat]
            shap_list.append(sub["SHAP_Value"].values)
            feature_list.append(sub["Feature_Value"].values)

        shap_matrix = np.array(shap_list).T
        feature_matrix = np.array(feature_list).T

        # --- FIGURE & AXES ---
        fig = plt.figure(figsize=(28, 22), dpi=300)
        PLOT_BOX = [0.14, 0.15, 0.55, 0.68]
        ax_main = fig.add_axes(PLOT_BOX)
        ax_top = ax_main.twiny()

        # --- DARK GREY BACKGROUND BARS ---
        y_pos = np.arange(num_features)
        
        # We draw the bars with height=0.85 so the beeswarm clusters fit inside
        ax_top.barh(
            y_pos,
            importance_vals_sorted,
            height=0.85,          # Wider bars to "envelop" the scatter points
            color="#555555",      # Darker charcoal grey
            edgecolor="none",     # Removed edges as requested
            alpha=0.25,           # Keeping alpha low so scatter points "pop"
            zorder=1
        )

        # --- SET AXIS RANGE TO 110% ---
        max_imp = np.max(importance_vals_sorted)
        ax_top.set_xlim(0, max_imp * 1.10) # 110% of maximum value

        ax_top.set_xlabel(
            "Global Feature Importance (Mean |SHAP|)",
            fontsize=46, fontweight="bold", labelpad=20, color="#555555"
        )

        ax_top.tick_params(
            axis="x", width=5, length=14, labelsize=40, labelcolor="#555555", pad=12 )

        ax_top.spines["top"].set_linewidth(8)
        ax_top.spines["top"].set_color("#555555")
        ax_top.set_zorder(1)

        # --- SHAP SUMMARY (BEESWARM) ---
        plt.sca(ax_main)
        shap.summary_plot(
            shap_matrix,
            feature_matrix,
            feature_names=sorted_features,
            show=False,
            cmap="viridis",
            alpha=0.45,
            plot_size=None
        )

        for obj in ax_main.collections:
            if isinstance(obj, plt.matplotlib.collections.PathCollection):
                obj.set_sizes([520])
                obj.set_edgecolor("black")
                obj.set_linewidth(2.2)
                obj.set_zorder(10)

        # --- COLORBAR ---
        for ax in fig.get_axes():
            if ax not in (ax_main, ax_top):
                ax.set_position([PLOT_BOX[0] + PLOT_BOX[2] - 0.09, PLOT_BOX[1], 0.028, PLOT_BOX[3]])
                ax.tick_params(labelsize=34, width=0, length=0)
                ax.set_ylabel("Feature Value", fontsize=48, fontweight="bold")

        # --- AXIS STYLING ---
        ax_main.tick_params(axis="x", width=6, length=20, labelsize=44, pad=10)
        ax_main.tick_params(axis="y", width=0, length=0, labelsize=46, pad=20)

        for spine in ["left", "bottom", "right", "top"]:
            ax_main.spines[spine].set_linewidth(8)
            ax_main.spines[spine].set_color("black")

        ax_main.patch.set_alpha(0)
        ax_main.set_zorder(10)

        ax_main.set_title(get_formal_name(t_col), fontsize=56, fontweight="bold", pad=30)
        ax_main.set_xlabel("SHAP Value (Impact on Prediction)", fontsize=48, fontweight="bold", labelpad=10)
        plt.setp(ax_main.get_yticklabels(), fontweight="bold")

        # --- SAVE ---
        plt.savefig(f"{shap_save_dir}/Integrated_SHAP_Standardized.tif", dpi=300, transparent=True)
        plt.close()

    except Exception as e:
        logging.error(f"❌ Error for {t_name}: {e}")


print("🚀 SHAP PLOTS GENERATED: Dark charcoal bars (height 0.85) and 110% Importance Range.")

🚀 SHAP PLOTS GENERATED: Dark charcoal bars (height 0.85) and 110% Importance Range.


# Parity Plots

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from scipy.stats import pearsonr
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings

warnings.filterwarnings("ignore")

# =============================================================================
# 1. TARGET CONFIGURATION (FOLDER + LABEL SEPARATED)
# =============================================================================
TARGETS = [
    {
        "col": "σ",
        "folder": "Electrical_Conductivity_σ_S_cm_",
        "label": "σ (S/cm)",
        "ticks": [0, 3000, 6000, 9000, 12000, 15000]
    },
    {
        "col": "κ",
        "folder": "Thermal_Conductivity_κ_W_m-K_",
        "label": "κ (W/m·K)",
        "ticks": [0, 1, 2, 3, 4, 5]
    },
    {
        "col": "S",
        "folder": "Seebeck_Coefficient_S_μV_K_",
        "label": "S (μV/K)",
        "ticks": [-1000, -500,  0,  500, 1000]
    },
    {
        "col": "zT",
        "folder": "Figure_of_Merit_zT",
        "label": "zT value",
        "ticks": [0, 0.1, 0.2, 0.3, 0.4]
    }
]


# =============================================================================
# 2. GLOBAL STYLE (STRICT PUBLICATION)
# =============================================================================
plt.rcParams.update({
    "font.family": "Arial",
    "font.weight": "bold",
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.linewidth": 8,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "xtick.labelsize": 32,
    "ytick.labelsize": 32,
    "savefig.transparent": True,
    "savefig.dpi": 300
})

FIG_SIZE = (11, 11)
SCATTER_SIZE = 600
SCATTER_ALPHA = 0.7
EDGE_WIDTH = 2

# =============================================================================
# 3. MAIN
# =============================================================================
def main():

    for target in TARGETS:

        target_folder = target["folder"]
        target_label = target["label"]
        ticks = target["ticks"]

        parity_dir = os.path.join(FIGURES_DIR, target_folder, "Parity")
        metrics_dir = os.path.join(FIGURES_DIR, target_folder, "Metrics")
        os.makedirs(parity_dir, exist_ok=True)
        os.makedirs(metrics_dir, exist_ok=True)

        optimized_dir = os.path.join(MAIN_ROOT_DIR, target_folder, "02_Optimized_Optuna")
        print(f"📂 Checking: {optimized_dir}")

        if not os.path.exists(optimized_dir):
            print("❌ Directory not found, skipping.")
            continue

        for file in os.listdir(optimized_dir):
            if not (file.startswith("parity_all_folds_") and file.endswith(".xlsx")):
                continue

            model_name = file.replace("parity_all_folds_", "").replace(".xlsx", "")
            file_path = os.path.join(optimized_dir, file)

            df = pd.read_excel(file_path)

            # =============================================================
            # METRICS (ALL FOLDS)
            # =============================================================
            metrics = []

            for fold in sorted(df["Fold"].unique()):
                fold_df = df[df["Fold"] == fold]
                y_true = fold_df["Actual"].values
                y_pred = fold_df["Predicted"].values

                metrics.append({
                    "Fold": fold,
                    "R2": r2_score(y_true, y_pred),
                    "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
                    "MAE": mean_absolute_error(y_true, y_pred),
                    "Pearson_r": pearsonr(y_true, y_pred)[0]
                })

            metrics_df = pd.DataFrame(metrics)
            summary = metrics_df.drop(columns="Fold").agg(["mean", "std"]).reset_index()
            summary.insert(0, "Fold", ["Mean", "Std"])
            final_metrics = pd.concat([metrics_df, summary], ignore_index=True)

            final_metrics.to_excel(
                os.path.join(metrics_dir, f"parity_metrics_{model_name}.xlsx"),
                index=False
            )

            # =============================================================
            # BEST FOLD (FOR VISUAL ONLY)
            # =============================================================
            best_fold = metrics_df.loc[metrics_df["R2"].idxmax(), "Fold"]
            best_df = df[df["Fold"] == best_fold]

            train_df = best_df[best_df["Set"] == "Train"]
            test_df = best_df[best_df["Set"] == "Test"]

            # =============================================================
            # PARITY PLOT
            # =============================================================
            fig, ax = plt.subplots(figsize=FIG_SIZE)

            ax.scatter(
                train_df["Actual"], train_df["Predicted"],
                s=SCATTER_SIZE, 
                c="r",
                marker="o",
               
                alpha=1,
                edgecolor="black", 
                linewidth=EDGE_WIDTH, 
                label="Train data", 
                zorder=4
            )

            ax.scatter(
                test_df["Actual"], test_df["Predicted"],
                s=SCATTER_SIZE, 
                c="y",
                marker="o",
                
                alpha=1,
                edgecolor="black",
                linewidth=EDGE_WIDTH, 
                label="Test data", zorder=5
            )

            
            tick_min, tick_max = min(ticks), max(ticks)
            range_ = tick_max - tick_min
            
            # 110% expansion (5% on each side)
            pad = 0.075 * range_
            
            plot_min = tick_min - pad
            plot_max = tick_max + pad
            
            # 1:1 line across full plot
            ax.plot(
                [plot_min+0.5*pad , plot_max-0.5*pad],
                [plot_min+ 0.5*pad, plot_max - 0.5*pad],
                color='red',
                linestyle="--",
                linewidth=5,
                zorder=2
            )
            
            # Expanded limits
            ax.set_xlim(plot_min, plot_max)
            ax.set_ylim(plot_min, plot_max)


            # IDENTICAL TICKS
            ax.set_xticks(ticks)
            ax.set_yticks(ticks)
            ax.set_xticklabels(ticks)
            ax.set_yticklabels(ticks)

            # Minor ticks
            ax.xaxis.set_minor_locator(AutoMinorLocator())
            ax.yaxis.set_minor_locator(AutoMinorLocator())

            # Grid
            ax.grid(True, which="major", linestyle="--", linewidth=2.5, alpha=0.6)
            ax.grid(True, which="minor", linestyle=":", linewidth=1.5, alpha=0.4)

            # Labels & title
            ax.set_xlabel(f"Actual {target_label}", fontsize=40, labelpad=5)
            ax.set_ylabel(f"Predicted {target_label}", fontsize=40, labelpad=5)
            ax.set_title(model_name, fontsize=38, pad=10)

            ax.set_aspect("equal", adjustable="box")

            ax.tick_params(which="major", length=18, width=5, pad=12)
            ax.tick_params(which="minor", length=10, width=3)

            for spine in ax.spines.values():
                spine.set_linewidth(8)

            ax.legend(
                loc="upper left",
                bbox_to_anchor=(0.015, 1),   # shift right & down from corner
                fontsize=26,
                frameon=True,
                framealpha=0.7,
                borderpad=0.6,
                labelspacing=0.5,
                markerscale=1
            )

            plt.tight_layout()
            plt.savefig(os.path.join(parity_dir, f"Parity_{model_name}.tif"))
            plt.close()

            print(f"✅ Saved parity & metrics for {model_name}")

    print("\n🎯 PARITY PLOTS & METRICS GENERATED SUCCESSFULLY")


# =============================================================================
if __name__ == "__main__":
    main()


📂 Checking: FINAL_RESULTS\Electrical_Conductivity_σ_S_cm_\02_Optimized_Optuna
✅ Saved parity & metrics for AdaBoost
✅ Saved parity & metrics for CatBoost
✅ Saved parity & metrics for ExtraTrees
✅ Saved parity & metrics for GradientBoost
✅ Saved parity & metrics for HistGradientBoosting
✅ Saved parity & metrics for LightGBM
✅ Saved parity & metrics for RandomForest
✅ Saved parity & metrics for XGBoost
📂 Checking: FINAL_RESULTS\Thermal_Conductivity_κ_W_m-K_\02_Optimized_Optuna
✅ Saved parity & metrics for AdaBoost
✅ Saved parity & metrics for CatBoost
✅ Saved parity & metrics for ExtraTrees
✅ Saved parity & metrics for GradientBoost
✅ Saved parity & metrics for HistGradientBoosting
✅ Saved parity & metrics for LightGBM
✅ Saved parity & metrics for RandomForest
✅ Saved parity & metrics for XGBoost
📂 Checking: FINAL_RESULTS\Seebeck_Coefficient_S_μV_K_\02_Optimized_Optuna
✅ Saved parity & metrics for AdaBoost
✅ Saved parity & metrics for CatBoost
✅ Saved parity & metrics for ExtraTrees
✅ Sa

In [44]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator


# =============================================================================
# 2. TARGET NAME MAPPING (CLEAN & SCIENTIFIC)
# =============================================================================
TARGET_LABELS = {
    "Electrical_Conductivity_σ_S_cm_": "σ (S/cm)",
    "Thermal_Conductivity_κ_W_m-K_": "κ (W/m-K)",
    "Seebeck_Coefficient_S_μV_K_": "S (μV/K)",
    "Figure_of_Merit_zT": "zT value"
}

# =============================================================================
# 3. FIND OPTUNA LOG FILES
# =============================================================================
optuna_logs = glob.glob(
    os.path.join(MAIN_ROOT_DIR, "**", "02_Optimized_Optuna", "optuna_log_*.xlsx"),
    recursive=True
)

print(f"🔍 Found {len(optuna_logs)} Optuna log files")

# =============================================================================
# 4. PLOT FUNCTION
# =============================================================================
def plot_optuna_convergence(excel_path):

    df = pd.read_excel(excel_path)

    if "Trial" not in df.columns or "Value" not in df.columns:
        print(f"❌ Skipping: {excel_path}")
        return

    df = df[["Trial", "Value"]].dropna().sort_values("Trial")

    trials = df["Trial"].values
    r2_values = df["Value"].values
    best_r2 = np.maximum.accumulate(r2_values)

    model_name = (
        os.path.basename(excel_path)
        .replace("optuna_log_", "")
        .replace(".xlsx", "")
    )

    target_folder = os.path.basename(
        os.path.dirname(os.path.dirname(excel_path))
    )
    target_label = TARGET_LABELS.get(target_folder, target_folder)

    save_dir = os.path.join(FIGURES_DIR, target_folder, "Optuna")
    os.makedirs(save_dir, exist_ok=True)

    save_path = os.path.join(
        save_dir,
        f"optuna_convergence_{model_name}.tif"
    )

    # =============================================================================
    # FIGURE
    # =============================================================================
    fig, ax = plt.subplots(figsize=(10, 8))
    fig.patch.set_alpha(0)
    ax.set_facecolor("none")

    # Curves
    ax.plot(
        trials, r2_values,
        color="grey",
        linewidth=0.5,
        alpha=0.6,
        label="Trial R² value"
    )
    ax.plot(
        trials, best_r2,
        color="dodgerblue",
        linewidth=5,
        label="Best R² value"
    )

    # Labels (bigger than ticks)
    ax.set_xlabel("Optuna Trial", fontsize=28, fontweight="bold", labelpad=5)
    ax.set_ylabel("R² Score", fontsize=28, fontweight="bold", labelpad=5)

    # Title (largest)
    ax.set_title(
        f"{target_label} | {model_name}",
        fontsize=28,
        fontweight="bold",
        pad=10
    )

    # Major ticks
    ax.tick_params(
        axis="both",
        which="major",
        labelsize=20,
        width=3,
        length=8,
        pad=10
    )

    # Minor ticks
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(
        axis="both",
        which="minor",
        width=1.5,
        length=4
    )

    # Spines (thick but spaced)
    for spine in ax.spines.values():
        spine.set_linewidth(4)

    # Grid
    ax.grid(True, which="major", linestyle="--", alpha=0.4)

    # Legend
    ax.legend(fontsize=18, frameon=True)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, transparent=True)
    plt.close()

    print(f"✅ Saved: {save_path}")

# =============================================================================
# 5. RUN ALL
# =============================================================================
for log_file in optuna_logs:
    try:
        plot_optuna_convergence(log_file)
    except Exception as e:
        print(f"❌ Failed for {log_file}: {e}")

print("\n🎯 ALL OPTUNA CONVERGENCE PLOTS GENERATED SUCCESSFULLY.")


🔍 Found 32 Optuna log files
✅ Saved: Figures\Electrical_Conductivity_σ_S_cm_\Optuna\optuna_convergence_AdaBoost.tif
✅ Saved: Figures\Electrical_Conductivity_σ_S_cm_\Optuna\optuna_convergence_CatBoost.tif
✅ Saved: Figures\Electrical_Conductivity_σ_S_cm_\Optuna\optuna_convergence_ExtraTrees.tif
✅ Saved: Figures\Electrical_Conductivity_σ_S_cm_\Optuna\optuna_convergence_GradientBoost.tif
✅ Saved: Figures\Electrical_Conductivity_σ_S_cm_\Optuna\optuna_convergence_HistGradientBoosting.tif
✅ Saved: Figures\Electrical_Conductivity_σ_S_cm_\Optuna\optuna_convergence_LightGBM.tif
✅ Saved: Figures\Electrical_Conductivity_σ_S_cm_\Optuna\optuna_convergence_RandomForest.tif
✅ Saved: Figures\Electrical_Conductivity_σ_S_cm_\Optuna\optuna_convergence_XGBoost.tif
✅ Saved: Figures\Figure_of_Merit_zT\Optuna\optuna_convergence_AdaBoost.tif
✅ Saved: Figures\Figure_of_Merit_zT\Optuna\optuna_convergence_CatBoost.tif
✅ Saved: Figures\Figure_of_Merit_zT\Optuna\optuna_convergence_ExtraTrees.tif
✅ Saved: Figures\Fi

In [52]:
# =============================================================================
# FINAL MASTER PLOTTING CELL — STRICTLY ORIGINAL STYLE PRESERVED
# =============================================================================

import os, re, glob, warnings, logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as ticker
from matplotlib.ticker import AutoMinorLocator
from matplotlib import cm
from scipy.stats import pearsonr
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import shap

warnings.filterwarnings("ignore")

# =============================================================================
# GLOBAL STYLE (UNCHANGED)
# =============================================================================
plt.rcParams.update({
    "font.family": "Arial",
    "font.weight": "bold",
    "axes.linewidth": 8.0,
    "xtick.direction": "out",
    "ytick.direction": "out",
    "savefig.transparent": True,
    "savefig.dpi": 300
})

MAIN_ROOT_DIR = "FINAL_RESULTS"
FIGURES_DIR = "Figures4"
os.makedirs(FIGURES_DIR, exist_ok=True)

# =============================================================================
# TARGETS
# =============================================================================
TARGETS = [
    {"name": "Electrical_Conductivity_σ_S_cm_", "col": "σ"},
    {"name": "Thermal_Conductivity_κ_W_m-K_", "col": "κ"},
    {"name": "Seebeck_Coefficient_S_μV_K_", "col": "S"},
    {"name": "Figure_of_Merit_zT", "col": "zT"}
]

TARGETS_PARITY = [
    {"col":"σ","folder":"Electrical_Conductivity_σ_S_cm_","label":"σ (S/cm)","ticks":[0,3000,6000,9000,12000,15000]},
    {"col":"κ","folder":"Thermal_Conductivity_κ_W_m-K_","label":"κ (W/m·K)","ticks":[0,1,2,3,4,5]},
    {"col":"S","folder":"Seebeck_Coefficient_S_μV_K_","label":"S (μV/K)","ticks":[-1000,-500,0,500,1000]},
    {"col":"zT","folder":"Figure_of_Merit_zT","label":"zT value","ticks":[0,0.1,0.2,0.3,0.4]}
]

PROPERTY_LABELS = {
    "σ":"Electrical Conductivity σ (S/cm)",
    "κ":"Thermal Conductivity κ (W/m·K)",
    "S":"Seebeck Coefficient S (μV/K)",
    "zT":"Figure of Merit (zT)"
}

TARGET_LABELS_OPTUNA = {
    "Electrical_Conductivity_σ_S_cm_":"σ (S/cm)",
    "Thermal_Conductivity_κ_W_m-K_":"κ (W/m-K)",
    "Seebeck_Coefficient_S_μV_K_":"S (μV/K)",
    "Figure_of_Merit_zT":"zT value"
}

def get_formal_name(c): return PROPERTY_LABELS.get(c, c)
def clean_filename(s): return re.sub(r"[^\w\-]", "_", str(s))

# =============================================================================
# CREATE OUTPUT DIRECTORIES
# =============================================================================
for t in TARGETS:
    for f in ["Analysis","Parity","Metrics","SHAP","Optuna"]:
        os.makedirs(f"{FIGURES_DIR}/{t['name']}/{f}", exist_ok=True)

# =============================================================================
# 1. PCC HEATMAP — SIZE RESTORED
# =============================================================================
def plot_pcc(target):
    name, col = target["name"], target["col"]
    df = pd.read_csv(f"{MAIN_ROOT_DIR}/{name}/processed_data_{name}.csv")
    rfe = pd.read_excel(f"{MAIN_ROOT_DIR}/{name}/00_Common_Analysis/RFE_Feature_Selection_Details.xlsx")

    feats = rfe.loc[rfe.Is_Selected, "Feature_Name"].tolist()
    feats = [f for f in feats if f in df.columns]

    corr = df[feats + [col]].corr()
    n = corr.shape[0]

    fig, ax = plt.subplots(figsize=(14, 13))  # ← ORIGINAL SIZE
    cmap = plt.get_cmap("RdYlGn")

    mask = np.triu(np.ones_like(corr, bool))
    im = ax.imshow(corr.where(~mask), cmap=cmap, vmin=-1, vmax=1)

    for i in range(n):
        for j in range(n):
            v = corr.iloc[i, j]
            if i > j:
                ax.add_patch(patches.Rectangle((j-0.5, i-0.5), 1, 1,
                                               fill=False, edgecolor="white", lw=5))
            elif i < j and np.isfinite(v):
                ax.pie([abs(v), 1-abs(v)],
                       colors=[cmap((v+1)/2), "white"],
                       radius=0.42, center=(j, i),
                       wedgeprops=dict(width=0.42, edgecolor="black", linewidth=2.8))
            elif i == j:
                ax.text(j, i, corr.columns[i], ha="center", va="center",
                        fontsize=26, color="red", weight="bold")

    ax.set_xticks([]); ax.set_yticks([])
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_ticks([-1, -0.5, 0, 0.5, 1])
    cbar.set_label("Pearson Correlation Coefficient", fontsize=28, weight="bold")
    cbar.ax.tick_params(labelsize=22)

    ax.set_title(get_formal_name(col), fontsize=42, pad=30)
    plt.tight_layout()
    plt.savefig(f"{FIGURES_DIR}/{name}/Analysis/PCC_Asymmetric_Recomputed.tif")
    plt.close()

# =============================================================================
# 2. RFE vs R² — LEGEND FIXED (ORIGINAL)
# =============================================================================
def plot_rfe(target):
    name, col = target["name"], target["col"]
    path = f"{MAIN_ROOT_DIR}/{name}/00_Common_Analysis/RFE_Plot_Data.xlsx"
    if not os.path.exists(path): return

    df = pd.read_excel(path)
    fig = plt.figure(figsize=(20, 12))
    ax = fig.add_axes([0.15, 0.15, 0.55, 0.7])

    ax.plot(df.Num_Features, df.CV_R2_Score,
            color=plt.get_cmap("RdYlGn")(0.9),
            marker="o", markersize=25,
            linewidth=8, markerfacecolor="white", markeredgewidth=5,
            label="CV R² Score")

    opt = int(df.loc[df.CV_R2_Score.idxmax(), "Num_Features"])
    ax.axvline(opt, color="red", linestyle="--", linewidth=6,
               label=f"Optimal: {opt} Features")

    ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())

    ax.tick_params(which="major", length=20, width=6, labelsize=40)
    ax.tick_params(which="minor", length=12, width=4)

    ax.set_xlabel("Number of Features", fontsize=40)
    ax.set_ylabel("Cross-Validation R² Score", fontsize=40)
    ax.set_title(get_formal_name(col), fontsize=44)

    # ORIGINAL LEGEND LOGIC
    if col == "κ":
        loc, bbox = "upper right", (0.98, 0.98)
    else:
        loc, bbox = "upper left", (0.02, 0.98)

    ax.legend(loc=loc, bbox_to_anchor=bbox,
              frameon=True, facecolor="white",
              edgecolor="black", framealpha=1,
              prop={"size":26, "weight":"bold"})

    plt.savefig(f"{FIGURES_DIR}/{name}/Analysis/RFE_Optimization.tif",
                bbox_inches="tight")
    plt.close()

# =============================================================================
# 3. OPTUNA CONVERGENCE — LEGEND + TITLE + ALL MODELS
# =============================================================================
def plot_optuna():
    logs = glob.glob(f"{MAIN_ROOT_DIR}/**/02_Optimized_Optuna/optuna_log_*.xlsx", recursive=True)

    for p in logs:
        df = pd.read_excel(p)
        if "Trial" not in df or "Value" not in df: continue

        df = df.sort_values("Trial")
        trials = df.Trial.values
        vals = df.Value.values
        best = np.maximum.accumulate(vals)

        model = os.path.basename(p).replace("optuna_log_", "").replace(".xlsx", "")
        target = os.path.basename(os.path.dirname(os.path.dirname(p)))
        label = TARGET_LABELS_OPTUNA.get(target, target)

        fig, ax = plt.subplots(figsize=(14, 9))
        ax.plot(trials, vals, color="grey", linewidth=1, alpha=0.6, label="Trial R²")
        ax.plot(trials, best, color="dodgerblue", linewidth=5, label="Best R²")

        ax.set_xlabel("Optuna Trial", fontsize=30)
        ax.set_ylabel("R² Score", fontsize=30)
        ax.set_title(f"{label} | {model}", fontsize=32, pad=15)

        ax.tick_params(which="major", labelsize=20, width=3, length=8)
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which="minor", length=4, width=1.5)

        ax.legend(fontsize=22, frameon=True)
        plt.tight_layout()
        plt.savefig(f"{FIGURES_DIR}/{target}/Optuna/optuna_convergence_{model}.tif")
        plt.close()

# =============================================================================
# 4. PARITY — TITLE, LABELS, GRID RESTORED
# =============================================================================
def plot_parity():
    for t in TARGETS_PARITY:
        folder = t["folder"]
        ticks = t["ticks"]
        path = f"{MAIN_ROOT_DIR}/{folder}/02_Optimized_Optuna"

        for f in os.listdir(path):
            if not f.startswith("parity_all_folds_"): continue
            model = f.replace("parity_all_folds_", "").replace(".xlsx", "")
            df = pd.read_excel(f"{path}/{f}")

            scores = [(fold, r2_score(df[df.Fold==fold].Actual,
                                       df[df.Fold==fold].Predicted))
                      for fold in df.Fold.unique()]
            best_fold = max(scores, key=lambda x:x[1])[0]
            d = df[df.Fold == best_fold]

            fig, ax = plt.subplots(figsize=(11,11))
            ax.scatter(d[d.Set=="Train"].Actual, d[d.Set=="Train"].Predicted,
                       s=600, c="#d73027", edgecolor="black", label="Train data")
            ax.scatter(d[d.Set=="Test"].Actual, d[d.Set=="Test"].Predicted,
                       s=600, c="#e1f297", edgecolor="black", label="Test data")

            pad = 0.075*(max(ticks)-min(ticks))
            lo, hi = min(ticks)-pad, max(ticks)+pad
            ax.plot([lo,hi], [lo,hi], "--", color="red", linewidth=5)

            ax.set_xlim(lo,hi); ax.set_ylim(lo,hi)
            ax.set_aspect("equal", "box")
            ax.grid(True, which="major", linestyle="--", linewidth=2.5, alpha=0.6)
            ax.grid(True, which="minor", linestyle=":", linewidth=1.5, alpha=0.4)

            ax.set_xticks(ticks); ax.set_yticks(ticks)
            ax.xaxis.set_minor_locator(AutoMinorLocator())
            ax.yaxis.set_minor_locator(AutoMinorLocator())

            ax.set_xlabel(f"Actual {t['label']}", fontsize=40)
            ax.set_ylabel(f"Predicted {t['label']}", fontsize=40)
            ax.set_title(model, fontsize=38)

            ax.legend(fontsize=26, frameon=True)
            plt.tight_layout()
            plt.savefig(f"{FIGURES_DIR}/{folder}/Parity/Parity_{model}.tif")
            plt.close()

# =============================================================================
# 5. SHAP — SUMMARY, DEPENDENCE, INTEGRATED (FULLY RESTORED)
# =============================================================================
def plot_shap(target):
    name, col = target["name"], target["col"]
    path = f"{MAIN_ROOT_DIR}/{name}/02_Optimized_Optuna/FINAL_RESULTS.xlsx"
    df = pd.read_excel(path, sheet_name="SHAP_Dependence_All")

    feats = df.Feature_Name.unique()
    shap_vals = np.column_stack([df[df.Feature_Name==f].SHAP_Value for f in feats])
    feat_vals = np.column_stack([df[df.Feature_Name==f].Feature_Value for f in feats])

    # SUMMARY
    plt.figure(figsize=(26, max(18, len(feats)*2.2)))
    shap.summary_plot(shap_vals, feat_vals, feature_names=feats,
                      cmap="viridis", show=False)
    plt.title(f"SHAP Summary: {get_formal_name(col)}", fontsize=56)
    plt.xlabel("SHAP Value (Impact on Prediction)", fontsize=56)
    plt.savefig(f"{FIGURES_DIR}/{name}/SHAP/Summary_Beeswarm_HD.tif")
    plt.close()

    # DEPENDENCE
    for f in feats:
        sub = df[df.Feature_Name==f]
        fig, ax = plt.subplots(figsize=(15,10.5))
        sc = ax.scatter(sub.Feature_Value, sub.SHAP_Value,
                        c=sub.Target_True_Value, cmap="viridis",
                        s=800, edgecolor="black", alpha=0.8)
        ax.axhline(0, linestyle="--", linewidth=6, color="black")
        ax.set_xlabel(f"Feature Value: {f}", fontsize=44)
        ax.set_ylabel("SHAP Value", fontsize=44)
        ax.set_title(f"Dependence: {f}", fontsize=52)
        cbar = plt.colorbar(sc, ax=ax)
        cbar.set_label(f"{col} value", fontsize=44)
        plt.savefig(f"{FIGURES_DIR}/{name}/SHAP/Dependence_{clean_filename(f)}.tif")
        plt.close()

    # INTEGRATED
    imp = df.groupby("Feature_Name").SHAP_Value.apply(lambda x:np.mean(abs(x))).sort_values()
    fig = plt.figure(figsize=(28,22))
    ax = fig.add_axes([0.14,0.15,0.55,0.68])
    ax2 = ax.twiny()
    ax2.barh(range(len(imp)), imp.values, height=0.85, color="#555555", alpha=0.25)
    shap.summary_plot(shap_vals, feat_vals, feature_names=feats,
                      cmap="viridis", show=False, alpha=0.45)
    ax.set_title(get_formal_name(col), fontsize=56)
    ax.set_xlabel("SHAP Value (Impact on Prediction)", fontsize=48)
    plt.savefig(f"{FIGURES_DIR}/{name}/SHAP/Integrated_SHAP_Standardized.tif")
    plt.close()

# =============================================================================
# RUN EVERYTHING
# =============================================================================
for t in TARGETS:
    plot_pcc(t)
    plot_rfe(t)
    plot_shap(t)

plot_optuna()
plot_parity()

print("✅ ALL PLOTS GENERATED — FULLY MATCHING ORIGINAL SETTINGS")


✅ ALL PLOTS GENERATED — FULLY MATCHING ORIGINAL SETTINGS
